Load the required packages

In [2]:
import pandas as pd
import numpy as np

*Load the raw dataset for rejected
loans*

In [3]:
data = pd.read_csv('rejected_2007_to_2017.lfs.csv')

*View the first rows of the dataset*

In [4]:
data.head()

,Amount Requested,Application Date,Loan Title,Risk_Score,Debt-To-Income Ratio,Zip Code,State,Employment Length,Policy Code
0,4000.0,2016-07-01,major_purchase,NaN,4.21%,750xx,TX,5 years,0.0
1,20000.0,2016-07-01,debt_consolidation,NaN,0.39%,930xx,CA,< 1 year,0.0
2,1000.0,2016-07-01,renewable_energy,NaN,42.38%,923xx,CA,5 years,0.0
3,15000.0,2016-07-01,home_improvement,NaN,5.98%,910xx,CA,5 years,0.0
4,8500.0,2016-07-01,debt_consolidation,NaN,29.14%,210xx,MD,< 1 year,0.0


*View the columns of the dataset*

In [5]:
data.columns

Index(['Amount Requested', 'Application Date', 'Loan Title', 'Risk_Score',
       'Debt-To-Income Ratio', 'Zip Code', 'State', 'Employment Length',
       'Policy Code'],
      dtype='object')

*Rename the columns to match the naming in the dataset of accepted loans. NOTE: application date is not really issue date, but they are assumed so in order to have a common time index between the two datasets.*

In [6]:
new_names = {'Amount Requested': 'loan_amnt', 'Loan Title': 'purpose', 'Debt-To-Income Ratio': 'dti',
             'Employment Length': 'emp_length', 'Application Date': 'issue_d', }

In [7]:
data.shape

(16131472, 9)

*Look at other columns in the dataset. First check for null values, then look at a sample of the data.*

In [8]:
data['Risk_Score'].isnull().sum()

9323619

In [9]:
data['Risk_Score'].sample(10)

3488888     705.0
1220145       NaN
16018728      NaN
12792043    774.0
9947905       NaN
4666752     701.0
7052831       NaN
9993777       NaN
10710788      NaN
11966417      NaN
Name: Risk_Score, dtype: float64

In [10]:
data['Policy Code'].astype(bool).sum(axis=0)

46029

*This feature is also quite sparse and is hence removed.*

*Drop not applicable columns such as state, zip code, policy code and risk score as not really a corresponding one in the accepted dataset...*

In [11]:
data.drop(['Risk_Score', 'Zip Code', 'State', 'Policy Code'], axis=1, inplace=True) #drop('issue_d', axis=1, inplace=True)

*Look at the remaining columns.*

In [12]:
data.columns

Index(['Amount Requested', 'Application Date', 'Loan Title',
       'Debt-To-Income Ratio', 'Employment Length'],
      dtype='object')

*Rename the columns as per the dictionary above.*

In [13]:
data = data.rename(mapper = new_names, axis = 1)

*Check column names now.*

In [14]:
data.columns

Index(['loan_amnt', 'issue_d', 'purpose', 'dti', 'emp_length'], dtype='object')

*Look at data formatting in the remaining columns to see what is yet to be formatted/cleaned.*

In [15]:
data.head()

,loan_amnt,issue_d,purpose,dti,emp_length
0,4000.0,2016-07-01,major_purchase,4.21%,5 years
1,20000.0,2016-07-01,debt_consolidation,0.39%,< 1 year
2,1000.0,2016-07-01,renewable_energy,42.38%,5 years
3,15000.0,2016-07-01,home_improvement,5.98%,5 years
4,8500.0,2016-07-01,debt_consolidation,29.14%,< 1 year


*Look at fractions of missing values per column.*

In [16]:
missing_fractions = data.isnull().mean().sort_values(ascending=False)

In [17]:
missing_fractions

emp_length    0.038865
purpose       0.000081
dti           0.000000
issue_d       0.000000
loan_amnt     0.000000
dtype: float64

*Format the 'emp_length' column to numbers from strings, as per the accepted dataset.*

In [18]:
data['emp_length'].replace(to_replace='10+ years', value='10 years', inplace=True)

In [19]:
data['emp_length'].replace('< 1 year', '0 years', inplace=True)

In [20]:
def emp_length_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])

In [21]:
data['emp_length'] = data['emp_length'].apply(emp_length_to_int)

*Check value counts.*

In [22]:
data['emp_length'].value_counts(dropna=False).sort_index()

 0.0     12479858
 1.0       134887
 2.0       120177
 3.0        98433
 4.0        71832
 5.0      2156621
 6.0        47255
 7.0        39351
 8.0        38755
 9.0        29748
 10.0      287601
NaN        626954
Name: emp_length, dtype: int64

*Set the rejected label for prediction, accepted will be 0 in the accepted dataset for a unique column when merged.*

In [23]:
data['rejected'] = 1

In [24]:
data.head()

,loan_amnt,issue_d,purpose,dti,emp_length,rejected
0,4000.0,2016-07-01,major_purchase,4.21%,5.0,1
1,20000.0,2016-07-01,debt_consolidation,0.39%,0.0,1
2,1000.0,2016-07-01,renewable_energy,42.38%,5.0,1
3,15000.0,2016-07-01,home_improvement,5.98%,5.0,1
4,8500.0,2016-07-01,debt_consolidation,29.14%,0.0,1


*Load the dataset of accepted loans.*

In [25]:
data_acc = pd.read_csv('accepted_2007_to_2017_all_V_0.lfs.csv')
data_acc.drop('Unnamed: 0', axis=1, inplace=True)

*View column labels*

In [26]:
data_acc.columns

Index(['loan_amnt', 'term', 'installment', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'purpose', 'dti', 'earliest_cr_line',
       'open_acc', 'pub_rec', 'revol_util', 'total_acc', 'application_type',
       'mort_acc', 'pub_rec_bankruptcies', 'log_annual_inc', 'fico_score',
       'log_revol_bal', 'charged_off'],
      dtype='object')

In [27]:
data_acc['rejected'] = 0

In [28]:
data_acc.columns

Index(['loan_amnt', 'term', 'installment', 'emp_length', 'home_ownership',
       'verification_status', 'issue_d', 'purpose', 'dti', 'earliest_cr_line',
       'open_acc', 'pub_rec', 'revol_util', 'total_acc', 'application_type',
       'mort_acc', 'pub_rec_bankruptcies', 'log_annual_inc', 'fico_score',
       'log_revol_bal', 'charged_off', 'rejected'],
      dtype='object')

*Columns to drop.*

In [29]:
to_drop = ['term', 'installment', 'home_ownership',
           'verification_status', 'earliest_cr_line',
           'open_acc', 'pub_rec', 'revol_util', 'total_acc', 'application_type',
           'mort_acc', 'pub_rec_bankruptcies', 'log_annual_inc', 'fico_score',
           'log_revol_bal', 'charged_off']

In [30]:
data_acc.drop(to_drop, axis=1, inplace=True)

In [31]:
data_acc.head()

,loan_amnt,emp_length,issue_d,purpose,dti,rejected
0,15000.0,10.0,Dec-2014,debt_consolidation,12.03,0
1,10400.0,8.0,Dec-2014,credit_card,14.92,0
2,21425.0,6.0,Dec-2014,credit_card,18.49,0
3,7650.0,0.0,Dec-2014,debt_consolidation,34.81,0
4,9600.0,10.0,Dec-2014,debt_consolidation,25.81,0


*Look at 'dti' column.*

In [32]:
data['dti'].describe()

count     16131472
unique       97768
top            -1%
freq       1106452
Name: dti, dtype: object

In [33]:
data_acc['dti'].describe()

count    814643.000000
mean         17.855847
std           8.367111
min           0.000000
25%          11.640000
50%          17.360000
75%          23.630000
max         197.590000
Name: dti, dtype: float64

In [34]:
new_data = data.copy()

*Turn into floats from percentage strings.*

In [35]:
new_data['dti'] = [float(i.rstrip("%")) for i in data["dti"]]

In [36]:
new_data['dti'].describe()

count    1.613147e+07
mean     1.725150e+02
std      1.372882e+04
min     -1.000000e+00
25%      6.480000e+00
50%      1.853000e+01
75%      3.438000e+01
max      5.000003e+07
Name: dti, dtype: float64

*Remove negative values as not meaningful.*

In [37]:
new_data = new_data.loc[new_data['dti'] >= 0]

In [38]:
new_data['dti'].describe()

count    1.502502e+07
mean     1.852927e+02
std      1.422526e+04
min      0.000000e+00
25%      9.140000e+00
50%      2.025000e+01
75%      3.580000e+01
max      5.000003e+07
Name: dti, dtype: float64

*View histogram of the distribution.*

In [39]:
import matplotlib.pyplot as plt
plt.figure()
plt.hist(new_data['dti'],range=[0, 100])
plt.show()

<Figure size 640x480 with 1 Axes>

In [40]:
new_data['dti'][(new_data['dti'] >= 200)].count()

285134

*Since these are not that many data points in comparison to the whole dataset, a cut out at 200% already seems reasonable...
This is roughly the upper limit in the accepted dataset*

*Look at the number of data points below 200%.*

In [41]:
new_data = new_data.loc[new_data['dti'] < 200]

In [42]:
new_data.shape

(14739886, 6)

In [43]:
data_acc.shape

(814643, 6)

In [44]:
new_data['issue_d'].describe()

count       14739886
unique          3781
top       2017-07-19
freq           30569
Name: issue_d, dtype: object

In [45]:
data_acc['issue_d'].describe()

count       814643
unique         124
top       Oct-2014
freq         33699
Name: issue_d, dtype: object

*Parse the column to dates.*

In [46]:
new_data['issue_d'] = pd.to_datetime(new_data['issue_d'])

In [47]:
new_data['issue_d'].describe()

count                14739886
unique                   3781
top       2017-07-19 00:00:00
freq                    30569
first     2007-05-26 00:00:00
last      2017-09-30 00:00:00
Name: issue_d, dtype: object

In [48]:
data_acc['issue_d'] = pd.to_datetime(data_acc['issue_d'])

In [49]:
data_acc['issue_d'].describe()

count                  814643
unique                    124
top       2014-10-01 00:00:00
freq                    33699
first     2007-06-01 00:00:00
last      2017-09-01 00:00:00
Name: issue_d, dtype: object

*Now just join the accepted and rejected datasets...*

In [ ]:
all_data = pd.concat([new_data, data_acc])

In [51]:
all_data.sample(5)

,dti,emp_length,issue_d,loan_amnt,purpose,rejected
1216126,18.92,10.0,2016-09-27,7000.0,Debt consolidation,1
10018985,33.88,5.0,2016-06-17,2250.0,medical,1
8381245,12.62,0.0,2017-08-23,4000.0,Credit card refinancing,1
8711611,34.79,0.0,2017-09-07,10000.0,Car financing,1
695741,19.42,1.0,2016-07-01,12000.0,debt_consolidation,0


*Check for counts of accepted and rejected loans to make sure the merger worked correctly.*

In [52]:
all_data['rejected'][(all_data['rejected'] == 0)].count()

814643

In [53]:
all_data['rejected'][(all_data['rejected'] == 1)].count()

14739886

*Save the data to an external file for checkpoint and get dummy variables.*

In [54]:
all_data.to_csv('all_data_accepted_and_rejected.lfs.csv', sep = ',')